---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [24]:
GDP = pd.read_excel(io = 'gdplev.xls', header=5)

In [25]:
GDP = GDP.iloc[2:,3:-1]
GDP.head()

,Unnamed: 3,GDP in billions of current dollars.1,GDP in billions of chained 2009 dollars.1
1929.0,1947q1,243.1,1934.5
1930.0,1947q2,246.3,1932.3
1931.0,1947q3,250.1,1930.3
1932.0,1947q4,260.3,1960.7
1933.0,1948q1,266.2,1989.5


In [26]:
GDP.rename(columns={"Unnamed: 3":"Quarter"}, inplace=True)

In [27]:
GDP.reset_index(inplace=True)

In [28]:
GDP.drop("index", axis = 1, inplace=True)

In [29]:
GDP[GDP['Quarter'] == "2000q1"]

,Quarter,GDP in billions of current dollars.1,GDP in billions of chained 2009 dollars.1
212,2000q1,10031.0,12359.1


In [30]:
GDP = GDP.drop(GDP.index[0:212])

In [31]:
homes = pd.read_csv('City_Zhvi_AllHomes.csv')
ut = pd.read_csv('university_towns.txt', sep="/br", header = None)

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


In [32]:
ut.columns = ['Original']

In [33]:
ut["RegionName"] = ut["Original"].apply(
    lambda x: x.partition('(')[0].rstrip(" ") if "[edit]" not in x else np.NaN)

In [34]:
ut["State"] = ut["Original"].apply(
    lambda x: x.partition('[')[0].strip(" ") if "[edit]" in x else np.NaN).fillna(method = "ffill")

In [14]:
ut = ut.dropna()

In [15]:
ut = ut[['State','RegionName']]

In [16]:
ut.head()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo


In [17]:
def get_list_of_university_towns():
    ut = pd.read_csv('university_towns.txt', sep="/br", header = None)
    ut.columns = ['Original']
    ut["RegionName"] = ut["Original"].apply(
        lambda x: x.partition('(')[0].rstrip(" ") if "[edit]" not in x else np.NaN)
    ut["State"] = ut["Original"].apply(
        lambda x: x.partition('[')[0].strip(" ") if "[edit]" in x else np.NaN).fillna(method = "ffill")
    ut = ut.dropna()
    ut = ut[['State','RegionName']]
    ut.reset_index(inplace=True, drop = True)
    return ut

In [38]:
GDP.reset_index(inplace = True, drop=True)

In [39]:
GDP

,Quarter,GDP in billions of current dollars.1,GDP in billions of chained 2009 dollars.1
0,2000q1,10031.0,12359.1
1,2000q2,10278.3,12592.5
2,2000q3,10357.4,12607.7
3,2000q4,10472.3,12679.3
4,2001q1,10508.1,12643.3
5,2001q2,10638.4,12710.3
6,2001q3,10639.5,12670.1
7,2001q4,10701.3,12705.3
8,2002q1,10834.4,12822.3
9,2002q2,10934.8,12893.0


In [40]:
GDP2 = GDP.shift(-1)
GDP2

,Quarter,GDP in billions of current dollars.1,GDP in billions of chained 2009 dollars.1
0,2000q2,10278.3,12592.5
1,2000q3,10357.4,12607.7
2,2000q4,10472.3,12679.3
3,2001q1,10508.1,12643.3
4,2001q2,10638.4,12710.3
5,2001q3,10639.5,12670.1
6,2001q4,10701.3,12705.3
7,2002q1,10834.4,12822.3
8,2002q2,10934.8,12893.0
9,2002q3,11037.1,12955.8


In [42]:
GDP3 = GDP2["GDP in billions of current dollars.1"]-GDP["GDP in billions of current dollars.1"]
i = list(GDP3[GDP3<0].index)
i

[31, 34, 35, 36]

In [44]:
GDP3[30:40]

30    115.6
31    -16.9
32    144.6
33     30.0
34   -293.1
35   -166.0
36    -43.5
37     43.7
38    182.4
39    114.6
Name: GDP in billions of current dollars.1, dtype: float64

In [68]:
x = len(i) 

In [95]:
def get_recession_start():
    GDP = pd.read_excel(io = 'gdplev.xls', header=5).iloc[2:,3:-1]
    GDP.rename(columns={"Unnamed: 3":"Quarter"}, inplace=True)
    GDP.reset_index(inplace=True,drop = True)
    GDP = GDP.drop(GDP.index[0:GDP[GDP['Quarter'] == "2000q1"].index.values])
    GDP.reset_index(inplace=True,drop = True)
    GDP2 = GDP.shift(-1)
    GDP3 = GDP2["GDP in billions of current dollars.1"]-GDP["GDP in billions of current dollars.1"]
    i = list(GDP3[GDP3<0].index)
    rece = []
    recelist = []
    for y in list(range(0,len(i))):
        if (i[y]-i[y-1] == 1) & (i[y-1] not in rece):
            rece.append(i[y-1])
    rece.append(i[y])
    for z in rece:
        recelist.append(GDP.loc[z][0])
    return recelist[0]

In [96]:
get_recession_start()

/opt/conda/lib/python3.6/site-packages/pandas/core/common.py:403: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  labels = list(labels)


'2008q3'

In [107]:
rece = []
recelist = []
for y in list(range(0,len(i))):
    if (i[y]-i[y-1] == 1) & (i[y-1] not in rece):
        rece.append(i[y-1])
rece.append(i[y])
rece.append(i[y]+2)

In [108]:
rece

[34, 35, 36, 38]

In [109]:
for z in rece:
    recelist.append(GDP.loc[z][0])

In [110]:
recelist

['2008q3', '2008q4', '2009q1', '2009q3']

In [119]:
def get_recession_end():
    GDP = pd.read_excel(io = 'gdplev.xls', header=5).iloc[2:,3:-1]
    GDP.rename(columns={"Unnamed: 3":"Quarter"}, inplace=True)
    GDP.reset_index(inplace=True,drop = True)
    GDP = GDP.drop(GDP.index[0:GDP[GDP['Quarter'] == "2000q1"].index.values])
    GDP.reset_index(inplace=True,drop = True)
    GDP2 = GDP.shift(-1)
    GDP3 = GDP2["GDP in billions of current dollars.1"]-GDP["GDP in billions of current dollars.1"]
    i = list(GDP3[GDP3<0].index)
    rece = []
    recelist = []
    for y in list(range(0,len(i))):
        if (i[y]-i[y-1] == 1) & (i[y-1] not in rece):
            rece.append(i[y-1])
    rece.append(i[y])
    rece.append(i[y]+3)
    for z in rece:
        recelist.append(GDP.loc[z][0])
    return recelist[-1]

In [121]:
get_recession_end()

/opt/conda/lib/python3.6/site-packages/pandas/core/common.py:403: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  labels = list(labels)


'2009q4'

In [122]:
def get_recession_bottom():
    GDP = pd.read_excel(io = 'gdplev.xls', header=5).iloc[2:,3:-1]
    GDP.rename(columns={"Unnamed: 3":"Quarter"}, inplace=True)
    GDP.reset_index(inplace=True,drop = True)
    GDP = GDP.drop(GDP.index[0:GDP[GDP['Quarter'] == "2000q1"].index.values])
    GDP.reset_index(inplace=True,drop = True)
    GDP2 = GDP.shift(-1)
    GDP3 = GDP2["GDP in billions of current dollars.1"]-GDP["GDP in billions of current dollars.1"]
    i = list(GDP3[GDP3<0].index)
    rece = []
    recelist = []
    for y in list(range(0,len(i))):
        if (i[y]-i[y-1] == 1) & (i[y-1] not in rece):
            rece.append(i[y-1])
    rece.append(i[y])
    rece.append(i[y]+2)
    return GDP.loc[i[y]+1][0]

In [123]:
get_recession_bottom()

/opt/conda/lib/python3.6/site-packages/pandas/core/common.py:403: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  labels = list(labels)


'2009q2'

In [217]:
homes = pd.read_csv('City_Zhvi_AllHomes.csv')

In [146]:
homes.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,17426,Chicago,IL,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4,50000.0,49900.0,49600.0,49400.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,40326,Phoenix,AZ,Phoenix,Maricopa,5,87200.0,87700.0,88200.0,88400.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


In [218]:
homes.set_index(['State','RegionName'], inplace = True)
homes = homes.loc[:,"2000-01":]
homes.head()

,,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
CA,Los Angeles,204400.0,207000.0,209800.0,212300.0,214500.0,216600.0,219000.0,221100.0,222800.0,224300.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
IL,Chicago,136800.0,138300.0,140100.0,141900.0,143700.0,145300.0,146700.0,147900.0,149000.0,150400.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
PA,Philadelphia,52700.0,53100.0,53200.0,53400.0,53700.0,53800.0,53800.0,54100.0,54500.0,54700.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
AZ,Phoenix,111000.0,111700.0,112800.0,113700.0,114300.0,115100.0,115600.0,115900.0,116500.0,117200.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


In [219]:
c1 = homes.columns.values.tolist()
c1

['2000-01',
 '2000-02',
 '2000-03',
 '2000-04',
 '2000-05',
 '2000-06',
 '2000-07',
 '2000-08',
 '2000-09',
 '2000-10',
 '2000-11',
 '2000-12',
 '2001-01',
 '2001-02',
 '2001-03',
 '2001-04',
 '2001-05',
 '2001-06',
 '2001-07',
 '2001-08',
 '2001-09',
 '2001-10',
 '2001-11',
 '2001-12',
 '2002-01',
 '2002-02',
 '2002-03',
 '2002-04',
 '2002-05',
 '2002-06',
 '2002-07',
 '2002-08',
 '2002-09',
 '2002-10',
 '2002-11',
 '2002-12',
 '2003-01',
 '2003-02',
 '2003-03',
 '2003-04',
 '2003-05',
 '2003-06',
 '2003-07',
 '2003-08',
 '2003-09',
 '2003-10',
 '2003-11',
 '2003-12',
 '2004-01',
 '2004-02',
 '2004-03',
 '2004-04',
 '2004-05',
 '2004-06',
 '2004-07',
 '2004-08',
 '2004-09',
 '2004-10',
 '2004-11',
 '2004-12',
 '2005-01',
 '2005-02',
 '2005-03',
 '2005-04',
 '2005-05',
 '2005-06',
 '2005-07',
 '2005-08',
 '2005-09',
 '2005-10',
 '2005-11',
 '2005-12',
 '2006-01',
 '2006-02',
 '2006-03',
 '2006-04',
 '2006-05',
 '2006-06',
 '2006-07',
 '2006-08',
 '2006-09',
 '2006-10',
 '2006-11',
 '20

In [220]:
Quarter = []
for i in c1:
    if i[-2:] in ["01","02","03"]:
        Quarter.append(i[:4]+'q1')
    elif i[-2:] in ["04","05","06"]:
        Quarter.append(i[:4]+'q2')
    elif i[-2:] in ["07","08","09"]:
        Quarter.append(i[:4]+'q3')
    elif i[-2:] in ["10","11","12"]:
        Quarter.append(i[:4]+'q4')
Quarter

['2000q1',
 '2000q1',
 '2000q1',
 '2000q2',
 '2000q2',
 '2000q2',
 '2000q3',
 '2000q3',
 '2000q3',
 '2000q4',
 '2000q4',
 '2000q4',
 '2001q1',
 '2001q1',
 '2001q1',
 '2001q2',
 '2001q2',
 '2001q2',
 '2001q3',
 '2001q3',
 '2001q3',
 '2001q4',
 '2001q4',
 '2001q4',
 '2002q1',
 '2002q1',
 '2002q1',
 '2002q2',
 '2002q2',
 '2002q2',
 '2002q3',
 '2002q3',
 '2002q3',
 '2002q4',
 '2002q4',
 '2002q4',
 '2003q1',
 '2003q1',
 '2003q1',
 '2003q2',
 '2003q2',
 '2003q2',
 '2003q3',
 '2003q3',
 '2003q3',
 '2003q4',
 '2003q4',
 '2003q4',
 '2004q1',
 '2004q1',
 '2004q1',
 '2004q2',
 '2004q2',
 '2004q2',
 '2004q3',
 '2004q3',
 '2004q3',
 '2004q4',
 '2004q4',
 '2004q4',
 '2005q1',
 '2005q1',
 '2005q1',
 '2005q2',
 '2005q2',
 '2005q2',
 '2005q3',
 '2005q3',
 '2005q3',
 '2005q4',
 '2005q4',
 '2005q4',
 '2006q1',
 '2006q1',
 '2006q1',
 '2006q2',
 '2006q2',
 '2006q2',
 '2006q3',
 '2006q3',
 '2006q3',
 '2006q4',
 '2006q4',
 '2006q4',
 '2007q1',
 '2007q1',
 '2007q1',
 '2007q2',
 '2007q2',
 '2007q2',
 '2007q3',

In [221]:
homes.columns = Quarter

In [189]:
homes.head()

,,2000q1,2000q1,2000q1,2000q2,2000q2,2000q2,2000q3,2000q3,2000q3,2000q4,...,2015q4,2015q4,2016q1,2016q1,2016q1,2016q2,2016q2,2016q2,2016q3,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
CA,Los Angeles,204400.0,207000.0,209800.0,212300.0,214500.0,216600.0,219000.0,221100.0,222800.0,224300.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
IL,Chicago,136800.0,138300.0,140100.0,141900.0,143700.0,145300.0,146700.0,147900.0,149000.0,150400.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
PA,Philadelphia,52700.0,53100.0,53200.0,53400.0,53700.0,53800.0,53800.0,54100.0,54500.0,54700.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
AZ,Phoenix,111000.0,111700.0,112800.0,113700.0,114300.0,115100.0,115600.0,115900.0,116500.0,117200.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


In [190]:
len(Quarter)

200

In [222]:
for x in range(0,len(Quarter)):
    if (Quarter[x] == Quarter[x-1]) and (Quarter[x] == Quarter[x-2]):
        homes[Quarter[x]] = homes[Quarter[x]].mean(axis = 1)
        

In [207]:
for x in range(0,len(Quarter)):
    if (Quarter[x] == Quarter[x+1]) and (Quarter[x] == Quarter[x+2]):
        print(homes[Quarter[x]])

                              2000q1     2000q1     2000q1
State RegionName                                          
NY    New York                   NaN        NaN        NaN
CA    Los Angeles           204400.0   207000.0   209800.0
IL    Chicago               136800.0   138300.0   140100.0
PA    Philadelphia           52700.0    53100.0    53200.0
AZ    Phoenix               111000.0   111700.0   112800.0
NV    Las Vegas             131700.0   132600.0   133500.0
CA    San Diego             219200.0   222900.0   226600.0
TX    Dallas                 85100.0    84500.0    83800.0
CA    San Jose              364100.0   374000.0   384700.0
FL    Jacksonville           88000.0    88800.0    89000.0
CA    San Francisco         418700.0   430600.0   442200.0
TX    Austin                142500.0   143000.0   143400.0
MI    Detroit                64400.0    66200.0    67900.0
OH    Columbus               93800.0    94400.0    94900.0
TN    Memphis                72400.0    72500.0    72600

[10730 rows x 3 columns]
                              2003q2     2003q2     2003q2
State RegionName                                          
NY    New York                   NaN        NaN        NaN
CA    Los Angeles           329600.0   334600.0   339300.0
IL    Chicago               191300.0   194100.0   197500.0
PA    Philadelphia           65700.0    66100.0    66800.0
AZ    Phoenix               137200.0   138000.0   138600.0
NV    Las Vegas             161700.0   162600.0   163700.0
CA    San Diego             374500.0   377600.0   380200.0
TX    Dallas                 98400.0    98500.0    98600.0
CA    San Jose              475400.0   473100.0   472600.0
FL    Jacksonville          112600.0   113000.0   114100.0
CA    San Francisco         574400.0   574700.0   577600.0
TX    Austin                154100.0   154200.0   154400.0
MI    Detroit                75000.0    75200.0    75400.0
OH    Columbus              110400.0   110800.0   111200.0
TN    Memphis                78

                              2006q2     2006q2     2006q2
State RegionName                                          
NY    New York              508800.0   515300.0   517000.0
CA    Los Angeles           585300.0   587300.0   589900.0
IL    Chicago               244900.0   245300.0   245400.0
PA    Philadelphia          110800.0   112500.0   113800.0
AZ    Phoenix               244800.0   245400.0   245600.0
NV    Las Vegas             297300.0   297800.0   298100.0
CA    San Diego             540100.0   539100.0   535900.0
TX    Dallas                118200.0   118600.0   118800.0
CA    San Jose              686000.0   685500.0   685000.0
FL    Jacksonville          170300.0   172600.0   174500.0
CA    San Francisco         783200.0   785300.0   785800.0
TX    Austin                182700.0   184100.0   185900.0
MI    Detroit                77800.0    77800.0    77900.0
OH    Columbus              122200.0   122200.0   122000.0
TN    Memphis                87000.0    87200.0    87300

                              2011q3     2011q3     2011q3
State RegionName                                          
NY    New York              469900.0   469500.0   468200.0
CA    Los Angeles           384700.0   382100.0   379500.0
IL    Chicago               175900.0   174100.0   172800.0
PA    Philadelphia          115300.0   115500.0   115600.0
AZ    Phoenix               104100.0   102900.0   102300.0
NV    Las Vegas             113000.0   112100.0   111400.0
CA    San Diego             371600.0   368900.0   366500.0
TX    Dallas                 94300.0    93900.0    93600.0
CA    San Jose              496100.0   495100.0   492300.0
FL    Jacksonville          107600.0   106600.0   105500.0
CA    San Francisco         672000.0   666300.0   662400.0
TX    Austin                199200.0   198200.0   197600.0
MI    Detroit                36500.0    36300.0    36300.0
OH    Columbus              101200.0    99900.0    98300.0
TN    Memphis                65700.0    65000.0    64700

IndexError: list index out of range

In [223]:
homes.head()

,,2000q1,2000q1,2000q1,2000q2,2000q2,2000q2,2000q3,2000q3,2000q3,2000q4,...,2015q4,2015q4,2016q1,2016q1,2016q1,2016q2,2016q2,2016q2,2016q3,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,572833.333333,572833.333333,582866.666667,582866.666667,582866.666667,591633.333333,591633.333333,591633.333333,588000,586400
CA,Los Angeles,207066.666667,207066.666667,207066.666667,214466.666667,214466.666667,214466.666667,220966.666667,220966.666667,220966.666667,226166.666667,...,557733.333333,557733.333333,566033.333333,566033.333333,566033.333333,577466.666667,577466.666667,577466.666667,583000,585100
IL,Chicago,138400.000000,138400.000000,138400.000000,143633.333333,143633.333333,143633.333333,147866.666667,147866.666667,147866.666667,152133.333333,...,207900.000000,207900.000000,206066.666667,206066.666667,206066.666667,208200.000000,208200.000000,208200.000000,211000,213000
PA,Philadelphia,53000.000000,53000.000000,53000.000000,53633.333333,53633.333333,53633.333333,54133.333333,54133.333333,54133.333333,54700.000000,...,122200.000000,122200.000000,123433.333333,123433.333333,123433.333333,126933.333333,126933.333333,126933.333333,128300,129100
AZ,Phoenix,111833.333333,111833.333333,111833.333333,114366.666667,114366.666667,114366.666667,116000.000000,116000.000000,116000.000000,117400.000000,...,183833.333333,183833.333333,187900.000000,187900.000000,187900.000000,191433.333333,191433.333333,191433.333333,194500,195900


In [232]:
home = homes.loc[:, ~homes.columns.duplicated()]

In [233]:
home

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,588000
CA,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,583000
IL,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,211000
PA,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128300
AZ,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,194500
NV,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199300
CA,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539000
TX,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,148200
CA,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,821700


In [234]:
def convert_housing_data_to_quarters():
    homes = pd.read_csv('City_Zhvi_AllHomes.csv')
    homes.set_index(['State','RegionName'], inplace = True)
    homes = homes.loc[:,"2000-01":]
    c1 = homes.columns.values.tolist()
    Quarter = []
    for i in c1:
        if i[-2:] in ["01","02","03"]:
            Quarter.append(i[:4]+'q1')
        elif i[-2:] in ["04","05","06"]:
            Quarter.append(i[:4]+'q2')
        elif i[-2:] in ["07","08","09"]:
            Quarter.append(i[:4]+'q3')
        elif i[-2:] in ["10","11","12"]:
            Quarter.append(i[:4]+'q4')
    homes.columns = Quarter
    for x in range(0,len(Quarter)):
        if (Quarter[x] == Quarter[x-1]) and (Quarter[x] == Quarter[x-2]):
            homes[Quarter[x]] = homes[Quarter[x]].mean(axis = 1)
    home = homes.loc[:, ~homes.columns.duplicated()]
    return home

In [235]:
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,588000
CA,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,583000
IL,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,211000
PA,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128300
AZ,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,194500
NV,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199300
CA,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539000
TX,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,148200
CA,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,821700


In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"